# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [417]:
import numpy as np
from scipy import optimize
import sympy as sm
from IPython.display import display
import matplotlib.pyplot as plt
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# The Basic Solow Model


\begin{equation}
Y_t = AK_t^{\alpha}L_t^{1-\alpha} \tag{1}
\end{equation}
\begin{equation}
S_t = sY_t \tag{2}
\end{equation}
\begin{equation}
K_{t+1} - K_t = S_t -\delta K_t \tag{3}
\end{equation}
\begin{equation}
L_{t+1} = (1+n)L_t \tag{4}
\end{equation}
\begin{equation}
y_t = \frac{Y_t}{L_t} = Ak_t^{\alpha} \tag{5}
\end{equation}
\begin{equation}
k_t = \frac{K_t}{L_t} \tag{6}
\end{equation}




In [418]:
Y = sm.symbols("Y_t")          #GDP in the economy
y = sm.symbols("y_t")          #GDP per worker
K = sm.symbols("K_t")          #Capital in the economy
K_next = sm.symbols("K_{t+1}") #Capital in the economy next period
k = sm.symbols("k_t")          #Capital per worker
k_next = sm.symbols("k_{t+1}") #Capital per worker
s = sm.symbols("s")            #Rate of savings
n = sm.symbols("n")            #Population growth
g = sm.symbols("g")            #Growth in technology
A = sm.symbols("A")            #Total factor productivity
L = sm.symbols("L_t")          #Labour
L_next = sm.symbols("L_{t+1}") #Labour next period
alpha = sm.symbols("alpha")    #Return on capital
delta = sm.symbols("delta")    #Depreciation rate on capital
kstar = sm.symbols("k^*")    #Steady state for capital
ystar = sm.symbols("y^*")      #Steady state for GDP per worker



In [419]:
r = sm.symbols("r_t")          #Real rental rate
w = sm.symbols("w_t")          #Real wage

\begin{equation}
r_t = \alpha A \left(\frac{K_t}{L_t}\right)^{\alpha-1} \tag{7}
\end{equation}
\begin{equation}
w_t = (1-\alpha) A \left(\frac{K_t}{L_t}\right)^{\alpha-1} \tag{8}
\end{equation}

In [420]:
##production_function = sm.Eq(A * K**(alpha) * L**(1-alpha), Y)

In [421]:
foc_wrt_K = sm.Eq(sm.simplify(sm.diff(A * K**(alpha) * L**(1-alpha), K)),r)
foc_wrt_K

Eq(A*K_t**(alpha - 1)*L_t**(1 - alpha)*alpha, r_t)

In [422]:
foc_wrt_L = sm.Eq(sm.simplify(sm.diff(A * K**(alpha) * L**(1-alpha), L)),w)
foc_wrt_L

Eq(-A*K_t**alpha*L_t**(-alpha)*(alpha - 1), w_t)

# Capital Accumulation

By inserting equation 1 in 2 and then inserting that in equation 3, finally we divide by equation 4 to end up at the transition equation

In [423]:
Transition_equation = sm.simplify(sm.Eq((((1/(1+n))*(A*s*k**(alpha)+(1-delta)*k))), k_next))

In [424]:
Transition_equation


Eq(k_{t+1}, (A*k_t**alpha*s - k_t*(delta - 1))/(n + 1))

To find the steady state of capital, we use that $$\begin{equation}
k_{t+1}=k_t=k^*
\end{equation}$$
 in steady state.


In [425]:
steady_state_capital = sm.solve(((A*s*kstar**(alpha)-(delta-1)*kstar)/(1+n)-kstar), kstar)
steady_state_capital[0]

((delta + n)/(A*s))**(1/(alpha - 1))

In [426]:
sm.Eq(steady_state_capital[0], kstar)

Eq(((delta + n)/(A*s))**(1/(alpha - 1)), k^*)

Using this result and using equation 5, we get the following steady state output

In [427]:
steady_state_output = sm.simplify(A*steady_state_capital[0]**alpha)
sm.Eq(ystar, steady_state_output)

Eq(y^*, A*(((delta + n)/(A*s))**(1/(alpha - 1)))**alpha)

In [428]:
steady_state_capital_func = sm.lambdify((delta,n,A,s,alpha),steady_state_capital[0])

def _steady_state_capital_func(delta,n,A,s,alpha):
    return steady_state_capital_func(delta,n,A,s,alpha)




In [429]:
delta= 0.05
n= 0.02
A= 1
savings_vec = np.array([0.1,0.2,0.3,0.4])
alpha = 0.33
steady_state_capital_func(delta,n,A,savings_vec,alpha)


array([ 1.70293062,  4.79176439,  8.77643682, 13.48323044])

In [448]:
def solow_graph(k,delta,n,A,s,alpha,t):
    # array of k values
    kvec = [k]
    # savings path
    svec = []
    # depreciation path
    deltavec = []


    for t_ in range(1,t):
        savings = s*A*t_**alpha
        #k_extended = t_ + savings - delta
        #depreciation_path = (delta+n)*t_
        svec.append(savings)
        #kvec.append(k_extended)
        #deltavec.append(depreciation_path)

    for t_ in range(1, t):
        savings = s*A*t_**alpha
        k_extended = t_ + savings - t_*delta
        kvec.append(k_extended)

    for t_ in range(1,t):
        depreciation_path = (delta+n)*t_
        deltavec.append(depreciation_path)


    plt.figure(figsize=(5,5))
    plt.plot(k_extended[:t], label='$sBk_t^{\u03B1}$', color = 'blue')
    plt.plot(depreciation_path[:t], label = '$(n+\delta)k_t$', color = 'black')
    plt.plot(savings[:t], label = '$(n+\delta)k_t$', color = 'red')
    plt.xlim(0,t)
    plt.ylim(0,depreciation_path[-1])
    plt.xlabel('$k_t$')
    plt.grid(True)
    plt.legend()
    return plt.show()
    


In [449]:
solow_graph(0,0.05,0.02,1,0.2,0.33,100)


TypeError: 'float' object is not subscriptable

In [432]:
def solow_graph(k,delta,n,A,s,alpha,t):
    # array of k values
    kvec = [k]
    # savings path
    svec = []
    # depreciation path
    deltavec = []


    for t_ in range(1,t):
        savings = s*A*t_**alpha
        k_extended = t_ + savings - delta
        depreciation_path = (delta+n)*t_
        svec.append(savings)
        kvec.append(k_extended)
        deltavec.append(depreciation_path)
    return kvec 

print(solow_graph(1,0.05,0.02,1,0.2,0.33,100))

[1, 1.15, 2.201402674904366, 3.2373955304369706, 4.266016524745351, 5.290165368616785, 6.311260025537067, 7.3301126726468375, 8.347236998174814, 9.362980954575736, 10.377592417900445, 11.391254941288128, 12.404108683780189, 13.416263373898072, 14.427806713342312, 15.438810032749542, 16.449332219560645, 17.45942251981927, 18.469122583324495, 19.47846798674529, 20.487489388144986, 21.49621341590576, 22.50466336277302, 23.51285973457709, 24.520820688998203, 25.528562390030967, 26.53609929703955, 27.5434444025033, 28.550609429107602, 29.557604994323587, 30.564440748766625, 31.571125493239066, 32.577667278317406, 33.58407348954718, 34.59035092069544, 35.596505837034115, 36.60254403025522, 37.6084708663243, 38.61429132734546, 39.62001004832384, 40.62563134956181, 41.63115926530262, 42.636597569136796, 43.64194979660477, 44.64721926536294, 45.65240909322484, 46.65752221434322, 47.662561393760704, 48.66752924052451, 49.672428219533614, 50.67726066226424, 51.68202877649999, 52.68673465517656, 5

In [433]:
print(svec)

[]
